Pipeline for Extracting Text from Aventura Joven Books

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tika

  Created wheel for tika: filename=tika-1.24-cp37-none-any.whl size=32885 sha256=da1f07d9bf0dc80df2a9e94106ff13519ac5af6cd7942f26247dee29291af2dc
  Stored in directory: /root/.cache/pip/wheels/73/9c/f5/0b1b738442fc2a2862bef95b908b374f8e80215550fb2a8975
Successfully built tika


In [36]:
#file_path = "/content/drive/MyDrive/capstone/Aventura Joven 03 - Perdidos en - Elvira Sancho, Jordi Suris.pdf"
file_path = "/content/drive/MyDrive/capstone/Aventura Joven 01 - Persecucion - Elvira Sancho, Jordi Suris.pdf"

In [37]:
# import parser object from tike
from tika import parser  
import unicodedata
  
# opening pdf file
parsed_pdf = parser.from_file(file_path)
  
# parsed_pdf['content'] returns string 
data = parsed_pdf['content'] 

In [ ]:
print(data[46000:47000].lower())

In [58]:
# Find beginning of text
beg_idx = data.lower().index('capítulo')
# Find end of text
end_idx = data.lower().index('después de la lectura\n')

print(beg_idx)
print(end_idx)

1771
46833


In [ ]:
# Check that this is indeed only the story text
text = data[beg_idx:end_idx]
print(data[beg_idx:end_idx])

In [61]:
# write the initial text to a file (no preprocessing done here)
with open('/content/drive/MyDrive/capstone/Aventura Joven 01 Text No Preproc.txt', 'w') as f:
  f.write(text)

In [62]:
print(text[:2000])

caPítulo 1

—¡Eh, Lucas!, ¡mira, allí!
—¿Dónde?
—Allí. ¿Ves a ese chico?
—¿Quién?
—Aquél, el de los vaqueros y las gafas. Allá, delante del super-

mercado.
Lucas mira con atención al chico. Después dice:
—No, ese chico no es. Es demasiado alto y no tiene el pelo 

negro.
—¡Ah!
—Además, tiene que entrar o salir de esta casa de ventanas 

verdes —Lucas señala una portería.
—El número veintitrés.
—Sí, el veintitrés.
Lucas y Rony están en la terraza de un bar de una placita1 en el 

barrio de La Latina2 de Madrid. Están bebiendo una cerveza y miran 
con atención a la gente que pasa. Lucas es alto, de nariz grande y 
ojos pequeños. Rony es bajo, un poco gordo y lleva una chaqueta 
vaquera azul.

�

1 placita: diminutivo de plaza, «pequeña plaza».
2  barrio de La Latina: barrio monumental de Madrid (cerca de la Plaza del Sol) 

que comprende parte del Madrid antiguo. Es un barrio muy acogedor en el que 
se encuentran muchos bares para ir de tapas.



—Voy a tomar otra cerveza —dice—. ¿Sabes

Preprocessing the text

In [63]:
# remove words with numbers in them, ex altura1 or hostal2 (footnote indicator)

import re

# remove special characters for regex
norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

text_wo_ft_words = norm_text
for word in re.findall('[A-z]+[1234567890]{1,2}', text_wo_ft_words):
  word_idx = text_wo_ft_words.index(word)
  m = re.search('[A-z]+', word)
  stripped_word = m.group(0)
  text_wo_ft_words = text_wo_ft_words[:word_idx] + stripped_word + text_wo_ft_words[word_idx+len(word):]
            
print(text_wo_ft_words[:2000])

caPitulo 1

Eh, Lucas!, mira, alli!
Donde?
Alli. Ves a ese chico?
Quien?
Aquel, el de los vaqueros y las gafas. Alla, delante del super-

mercado.
Lucas mira con atencion al chico. Despues dice:
No, ese chico no es. Es demasiado alto y no tiene el pelo 

negro.
Ah!
Ademas, tiene que entrar o salir de esta casa de ventanas 

verdes Lucas senala una porteria.
El numero veintitres.
Si, el veintitres.
Lucas y Rony estan en la terraza de un bar de una placita en el 

barrio de La Latina de Madrid. Estan bebiendo una cerveza y miran 
con atencion a la gente que pasa. Lucas es alto, de nariz grande y 
ojos pequenos. Rony es bajo, un poco gordo y lleva una chaqueta 
vaquera azul.



1 placita: diminutivo de plaza, pequena plaza.
2  barrio de La Latina: barrio monumental de Madrid (cerca de la Plaza del Sol) 

que comprende parte del Madrid antiguo. Es un barrio muy acogedor en el que 
se encuentran muchos bares para ir de tapas.



Voy a tomar otra cerveza dice. Sabes, Lucas? Con este...
Mira,

In [17]:
#with open('/content/drive/MyDrive/capstone/Aventura Joven 03 Text No Footnoted Words.txt', 'w') as f:
 # f.write(text_wo_ft_words)

In [ ]:
# remove footnote numbers on their own in the text

In [65]:
print(text_wo_ft_words[:200])
text_wo_ft_words[:200]

caPitulo 1

Eh, Lucas!, mira, alli!
Donde?
Alli. Ves a ese chico?
Quien?
Aquel, el de los vaqueros y las gafas. Alla, delante del super-

mercado.
Lucas mira con atencion al chico. Despues dice:
No, e


'caPitulo 1\n\nEh, Lucas!, mira, alli!\nDonde?\nAlli. Ves a ese chico?\nQuien?\nAquel, el de los vaqueros y las gafas. Alla, delante del super-\n\nmercado.\nLucas mira con atencion al chico. Despues dice:\nNo, e'

In [ ]:
def modify_words(regex_find, regex_modify, text, modify):
  ''' Modifies all words in text that match the regex_find regex to match the regex_modify regex. '''
  text_removed = text
  for word in re.findall(regex_find, text_removed):
    word_idx = text_removed.index(word)
    
    if modify == 'regex':
      m = re.search(regex_modify, word)
      modified_word = m.group(0)
    elif modify == 'remove':
      rm_char_idx = word.index(regex_modify)
      modified_word = word[:rm_char_idx] + word[rm_char_idx+len(regex_modify):]
    
    text_removed = text_removed[:word_idx] + modified_word + text_removed[word_idx+len(word):]

  return text_removed

In [ ]:
# remove words that contain '-\n' because they didn't fit on one line
text_wo_broken_words = text_wo_ft_words
for word in re.findall('[A-z]+\-\n{1,2}[a-z]+', text_wo_broken_words):
  print(word)
  print('######')
  word_idx = text_wo_broken_words.index(word)
  hyphen_idx = word.index('-')
  if '-\n\n' in word:
    len_sep = 3
  else:
    len_sep = 2
  modified_word = word[:hyphen_idx] + word[hyphen_idx+len_sep:]
  print(modified_word)
  print('-------------------------------')
  print(text_wo_broken_words[word_idx-50:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):word_idx+len(word)+50])
  text_wo_broken_words = text_wo_broken_words[:word_idx] + modified_word + text_wo_broken_words[word_idx+len(word):]

print(text_wo_broken_words[:1000])

In [68]:
with open('/content/drive/MyDrive/capstone/Aventura Joven 01 Text No Ft Broken Words.txt', 'w') as f:
  f.write(text_wo_broken_words)

In [ ]:
text_wo_broken_words = modify_words('[A-z]+\-\n[a-z]+', '-\n', text_wo_ft_words, 'remove')
print(text_wo_broken_words[:1000])

In [ ]:
# remove the footnotes

In [ ]:
# Attempt at removing the footnotes
import re

# normalize the data to ignore special characters in Spanish
norm_data = unicodedata.normalize('NFD', data).encode('ascii', 'ignore').decode('utf-8')
norm_text = unicodedata.normalize('NFD', text).encode('ascii', 'ignore').decode('utf-8')

m = re.search('[A-z]+[1234567890]{1,2}', norm_data[1880:])

m.group(0)
footnoted_words = [word.strip() for word in re.findall('[A-z]+[1234567890]', norm_data[1880:])]
footnote_defs = [word.strip() for word in re.findall('[0-9]{1,2}  [A-z]+[\s\w]*:[\w \n]*\.', norm_data[1880:])]

#print(footnoted_words)
#print(footnote_defs)
print(re.findall('([0-9]{1,2}  [A-z]+[\s\w]*:([\w ,\.:\-]+\n)*\n)', '''1  altura: Cusco esta a unos 3500 metros de altura, lo que en algunas personas 
provoca el llamado mal de altura o soroche. Los sintomas del mal de altura 
son: dolor de cabeza, mareos, trastornos estomacales y cansancio. Puede com-
batirse con pastillas, ejercicios de respiracion o mate.

2  hostal: alojamiento normalmente mas barato y personal que un hotel, aunque 
sin sus comodidades. La palabra se utiliza principalmente en medios rurales. 
'''))